In [ ]:
!nvidia-smi

Fri Jul 12 14:51:56 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P0              28W /  70W |   8423MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd

In [ ]:
dataset = pd.read_csv('/content/drive/MyDrive/train_10000.csv')

# Reduce the dataset to only 1000 rows
# dataset = dataset.head(1000)

dataset.shape

(10000, 3)

In [ ]:
dataset.head()

,headline,summary,article
0,"उत्तर भारत में शीत लहर जारी, लेह में पारा सात ...",NaN,उत्तर भारत में शीत लहर जारी है और पहाड़ी क्षेत...
1,यमन : विद्रोहियों के कब्जे वाले ताएज शहर के नि...,NaN,यमन सरकार के समर्थक बलों के विद्रोहियों के कब्...
2,केन्या में आतंकियों ने बस को अगवा कर 28 लोगों ...,NaN,"केन्या के उत्तर में, सोमालिया के अल-शबाब गुट क..."
3,रिम करेगा दो हजार कर्मचारियों की छंटनी,ब्लैकबेरी हैंडसेट बनाने वाली कनाडा की कंपनी रि...,ब्लैकबेरी हैंडसेट बनाने वाली कनाडा की कंपनी रि...
4,Puducherry: उपराज्यपाल किरण बेदी से हुआ विवाद ...,NaN,केंद्र शासित प्रदेश पुडुचेरी के मुख्यमंत्री व...


In [ ]:
dataset['summary'].isnull().sum()

5290

In [ ]:
dataset_cleaned = dataset[dataset['summary'].notnull()]
dataset_cleaned = dataset_cleaned[dataset_cleaned['article'].notnull()]
dataset_cleaned = dataset_cleaned[dataset_cleaned['headline'].notnull()]
dataset_cleaned.head()

,headline,summary,article
3,रिम करेगा दो हजार कर्मचारियों की छंटनी,ब्लैकबेरी हैंडसेट बनाने वाली कनाडा की कंपनी रि...,ब्लैकबेरी हैंडसेट बनाने वाली कनाडा की कंपनी रि...
7,अमेरिका : राष्ट्रपति पद की उम्मीदवार हिलेरी क्...,श्रद्धांजलि सभा के दौरान अचानक खराब हुई हिलेरी...,अमेरिका में राष्ट्रपति पद के लिए डेमोक्रेटिक प...
8,देश के सर्वश्रेष्ठ स्पिनरों में से एक है हरभजन...,पूर्व भारतीय कप्तान अनिल कुंबले का मानना है कि...,पूर्व भारतीय क्रिकेट कप्तान अनिल कुंबले का मान...
9,इंडिगो के स्टाफ ने दिल्ली एयरपोर्ट पर पैसेंजर ...,इंडिगो के ग्राउंड स्टाफ ने यात्री से की हाथापा...,इंडिगो एयरलाइन के प्रेसीडेंट और डायरेक्टर आदित...
10,नव्या नवेली के साथ दोस्ती पर जावेद जाफरी के बे...,मीजान जाफरी ने किया खुलासा\r\nनव्या नंदा को बत...,संजय लीला भंसाली की फिल्म 'मलाल' से बॉलीवुड मे...


In [ ]:
dataset_cleaned.shape

(4570, 3)

In [ ]:
dataset_cleaned['summary'].isnull().sum()

0

In [ ]:
dataset_cleaned['article'].isnull().sum()

0

In [ ]:
dataset_cleaned['headline'].isnull().sum()

0

In [ ]:
articles = list(dataset_cleaned['article'])
summaries = list(dataset_cleaned['summary'])
headlines = list(dataset_cleaned['headline'])

In [ ]:
from spacy.lang.hi import STOP_WORDS as STOP_WORDS_HI

In [ ]:
import re
hindi_pattern = r'[\u0965\u002E\u002C\u0021\u003F\u003B\u003A\u0027\u0022\u2018\u2019\u201C\u201D\u002D\u0028\u0029]'
purna_viram_pattern = r'[\u0964]'
english_pattern = r'[^\w\s\u0900-\u097F]'
pattern = hindi_pattern + '|' + english_pattern
article_corpus = []
summary_corpus = []
headline_corpus = []

In [ ]:
for i in range(len(articles)):
  sentence = re.sub(pattern, ' ', articles[i])
  sentence = re.sub(purna_viram_pattern, '.', sentence)
  sentence = sentence.split()
  sentence = [word for word in sentence if not word in STOP_WORDS_HI]
  sentence = ' '.join(sentence)
  article_corpus.append(sentence)

for i in range(len(summaries)):
  sentence = re.sub(pattern, ' ', summaries[i])
  sentence = re.sub(purna_viram_pattern, '.', sentence)
  sentence = sentence.split()
  sentence = [word for word in sentence] # Not removing stopwords from the summary
  sentence = ' '.join(sentence)
  summary_corpus.append(sentence)

for i in range(len(headlines)):
  sentence = re.sub(pattern, ' ', headlines[i])
  sentence = re.sub(purna_viram_pattern, '.', sentence)
  sentence = sentence.split()
  sentence = [word for word in sentence if not word in STOP_WORDS_HI]
  sentence = ' '.join(sentence)
  headline_corpus.append(sentence)

In [ ]:
import pandas as pd

preprocessed_df = pd.DataFrame({
    'article': article_corpus,
    'summary': summary_corpus,
    'headline': headline_corpus
})

preprocessed_df.head()

,article,summary,headline
0,ब्लैकबेरी हैंडसेट बनाने वाली कनाडा कंपनी रिसर्...,ब्लैकबेरी हैंडसेट बनाने वाली कनाडा की कंपनी रि...,रिम करेगा हजार कर्मचारियों छंटनी
1,अमेरिका राष्ट्रपति पद डेमोक्रेटिक पार्टी उम्मी...,श्रद्धांजलि सभा के दौरान अचानक खराब हुई हिलेरी...,अमेरिका राष्ट्रपति पद उम्मीदवार हिलेरी क्लिंटन...
2,पूर्व भारतीय क्रिकेट कप्तान अनिल कुंबले मानना ...,पूर्व भारतीय कप्तान अनिल कुंबले का मानना है कि...,देश सर्वश्रेष्ठ स्पिनरों हरभजन सिंह अनिल कुंबले
3,इंडिगो एयरलाइन प्रेसीडेंट डायरेक्टर आदित्य घोष...,इंडिगो के ग्राउंड स्टाफ ने यात्री से की हाथापा...,इंडिगो स्टाफ दिल्ली एयरपोर्ट पैसेंजर मारपीट एय...
4,संजय लीला भंसाली फिल्म मलाल बॉलीवुड कदम रखने म...,मीजान जाफरी ने किया खुलासा नव्या नंदा को बताया...,नव्या नवेली दोस्ती जावेद जाफरी बेटे खुलासा कही...


In [ ]:
preprocessed_df.shape

(4570, 3)

## **Model Importing and Fine-Tuning**

In [ ]:
# !pip install transformers
# !pip install tensorflow
# !pip install datasets
# !pip install pandas

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
# Split the dataset into training and validation sets
train_df, val_df = train_test_split(preprocessed_df, test_size=0.1)

# Save the splits to new CSV files
train_df.to_csv('train.csv', index=False)
val_df.to_csv('val.csv', index=False)

In [ ]:
# !pip install datasets

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer

# Load the dataset
data_files = {"train": "train.csv", "validation": "val.csv"}
datasets = load_dataset('csv', data_files=data_files)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("google/mt5-small")

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer

# Load the dataset
data_files = {"train": "train.csv", "validation": "val.csv"}
datasets = load_dataset('csv', data_files=data_files)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("google/mt5-small")

def preprocess_data(examples):
    inputs = ["summarize: " + doc for doc in examples["article"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=150, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Tokenize the datasets
tokenized_datasets = datasets.map(preprocess_data, batched=True)

Map:   0%|          | 0/4113 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/457 [00:00<?, ? examples/s]

In [ ]:
import tensorflow as tf
from transformers import TFAutoModelForSeq2SeqLM

# Load the model
model = TFAutoModelForSeq2SeqLM.from_pretrained("google/mt5-small")

# Define the data collator
def data_collator(features):
    input_ids = [feature['input_ids'] for feature in features]
    attention_mask = [feature['attention_mask'] for feature in features]
    labels = [feature['labels'] for feature in features]
    return {
        'input_ids': tf.ragged.constant(input_ids).to_tensor(default_value=0),
        'attention_mask': tf.ragged.constant(attention_mask).to_tensor(default_value=0),
        'labels': tf.ragged.constant(labels).to_tensor(default_value=-100)
    }

# Prepare the datasets
train_dataset = tokenized_datasets['train'].to_tf_dataset(
    columns=['input_ids', 'attention_mask', 'labels'],
    shuffle=True,
    batch_size=4,
    collate_fn=data_collator,
)

val_dataset = tokenized_datasets['validation'].to_tf_dataset(
    columns=['input_ids', 'attention_mask', 'labels'],
    shuffle=False,
    batch_size=4,
    collate_fn=data_collator,
)

/usr/local/lib/python3.10/dist-packages/tf_keras/src/initializers/initializers.py:121: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(
All model checkpoint layers were used when initializing TFMT5ForConditionalGeneration.

All the layers of TFMT5ForConditionalGeneration were initialized from the model checkpoint at google/mt5-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMT5ForConditionalGeneration for predictions without further training.


In [ ]:
# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
model.compile(optimizer=optimizer)

In [ ]:
hist = model.fit(train_dataset, validation_data=val_dataset, epochs=3)

In [ ]:
# Train the model
hist = model.fit(train_dataset, validation_data=val_dataset, epochs=2)

Epoch 1/2
1029/1029 [==============================] - 378s 368ms/step - loss: 4.4256 - val_loss: 2.4837
Epoch 2/2
1029/1029 [==============================] - 386s 375ms/step - loss: 4.0697 - val_loss: 2.3747


In [ ]:
model.summary()

Model: "tfmt5_for_conditional_generation_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 shared (Embedding)          multiple                  128057344 
                                                                 
 encoder (TFT5MainLayer)     multiple                  146940608 
                                                                 
 decoder (TFT5MainLayer)     multiple                  153236160 
                                                                 
 lm_head (Dense)             multiple                  128057344 
                                                                 
Total params: 300176768 (1.12 GB)
Trainable params: 300176768 (1.12 GB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
def summarize_text(text, max_length=150, min_length=40, num_beams=4):
    # Prepend the task prefix "summarize: " to the input text
    input_text = "summarize: " + text

    # Tokenize the input text
    input_ids = tokenizer.encode(input_text, return_tensors="tf", max_length=512, truncation=True)

    # Generate the summary
    summary_ids = model.generate(input_ids, max_length=max_length, min_length=min_length, num_beams=num_beams, length_penalty=2.0, early_stopping=True)

    # Decode the summary and return it
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

In [ ]:
hindi_text = "भारत, आधिकारिक रूप से भारत गणराज्य, दक्षिण एशिया का एक देश है। यह क्षेत्रफल के आधार पर सातवां सबसे बड़ा देश है, और जनसंख्या के आधार पर दूसरा सबसे बड़ा देश है। भारत का इतिहास बहुत पुराना है, जो सिंधु घाटी सभ्यता से शुरू होता है। भारतीय संस्कृति, धर्म, दर्शन, कला और विज्ञान के क्षेत्र में भारत का महत्वपूर्ण योगदान है। भारत एक बहुभाषी, बहुजातीय और बहुधार्मिक समाज है, जिसमें हिंदू धर्म, इस्लाम, ईसाई धर्म, सिख धर्म, बौद्ध धर्म, जैन धर्म और कई अन्य धार्मिक परंपराएं विद्यमान हैं। भारत की राजधानी नई दिल्ली है, और इसकी आधिकारिक भाषाएँ हिंदी और अंग्रेजी हैं। भारत की अर्थव्यवस्था दुनिया की सबसे तेज़ी से बढ़ती हुई अर्थव्यवस्थाओं में से एक है। कृषि, उद्योग और सेवाओं में इसकी विविधता और मजबूती इसे विश्व के प्रमुख आर्थिक केंद्रों में शामिल करती है। भारत के प्रमुख शहरों में मुंबई, दिल्ली, बेंगलुरु, चेन्नई, कोलकाता और हैदराबाद शामिल हैं। भारतीय समाज में परिवार, शिक्षा, और संस्कृति को अत्यधिक महत्व दिया जाता है। भारत के त्योहार, जैसे दिवाली, होली, ईद, क्रिसमस, और बैसाखी, राष्ट्रीय और सांस्कृतिक एकता को दर्शाते हैं। विभिन्न भाषाओं, धार्मिक परंपराओं, और सांस्कृतिक विरासतों के बावजूद, भारत एकता में विविधता का जीवंत उदाहरण है।"

# Generate summary
summary = summarize_text(hindi_text)
print("Summary:", summary)

Summary: सांस्कृतिक परंपराओं में शामिल हैं और राष्ट्रीय और सांस्कृतिक परंपराओं में शामिल हैं और इसके प्रमुख शहरों में शामिल हैं और भारत के प्रमुख शहरों में शामिल हैं और इसके प्रमुख शहरों में शामिल हैं और इसके प्रमुख शहरों में शामिल हैं और राष्ट्रीय और सांस्कृतिक परंपराओं में शामिल हैं।


In [ ]:
# !pip install rouge-score

In [ ]:
# Define the path where you want to save the model in Google Drive
model_save_path = '/content/drive/MyDrive/fine_tuned_model_epoch_5'

# Save the model to the local filesystem
model.save_pretrained("./fine_tuned_model_epoch_5")
tokenizer.save_pretrained("./fine_tuned_model_epoch_5")

# Copy the model files to Google Drive
!cp -r ./fine_tuned_model_epoch_5 /content/drive/MyDrive/

In [ ]:
# !pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 1.4 MB/s eta 0:00:00


In [ ]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

In [ ]:
sampled_val_df = val_df.sample(frac=0.05, random_state=42)

In [ ]:
# Load the model and tokenizer from the saved directory
model = TFAutoModelForSeq2SeqLM.from_pretrained("./fine_tuned_model_epoch_5")
tokenizer = AutoTokenizer.from_pretrained("./fine_tuned_model_epoch_5")

/usr/local/lib/python3.10/dist-packages/tf_keras/src/initializers/initializers.py:121: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(
All model checkpoint layers were used when initializing TFMT5ForConditionalGeneration.

All the layers of TFMT5ForConditionalGeneration were initialized from the model checkpoint at ./fine_tuned_model_epoch_5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMT5ForConditionalGeneration for predictions without further training.
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemen

In [ ]:
# Sample 5% of the validation dataset
sampled_val_df = val_df.sample(frac=0.05, random_state=42)

def summarize_text(text, max_length=150, min_length=40, num_beams=4):
    # Prepend the task prefix "summarize: " to the input text
    input_text = "summarize: " + text

    # Tokenize the input text
    input_ids = tokenizer.encode(input_text, return_tensors="tf", max_length=512, truncation=True)

    # Generate the summary
    summary_ids = model.generate(input_ids, max_length=max_length, min_length=min_length, num_beams=num_beams, length_penalty=2.0, early_stopping=True)

    # Decode the summary and return it
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# Generate summaries for the sampled validation dataset
generated_summaries = []
for article in sampled_val_df['article']:
    generated_summary = summarize_text(article)
    generated_summaries.append(generated_summary)


# Add generated summaries to the sampled validation dataset DataFrame
sampled_val_df['generated_summary'] = generated_summaries

# Initialize BLEU scorer
smoothie = SmoothingFunction().method4

# Initialize ROUGE scorer
rouge_scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Calculate BLEU and ROUGE scores for each pair of summaries
bleu_scores = []
rouge1_scores = []
rouge2_scores = []
rougeL_scores = []

for index, row in sampled_val_df.iterrows():
    generated_summary = row['generated_summary']
    reference_summary = row['summary']

    # Calculate BLEU score
    bleu_score = sentence_bleu([reference_summary.split()], generated_summary.split(), smoothing_function=smoothie)
    bleu_scores.append(bleu_score)

    # Calculate ROUGE scores
    rouge_score = rouge_scorer.score(reference_summary, generated_summary)
    rouge1_scores.append(rouge_score['rouge1'].fmeasure)
    rouge2_scores.append(rouge_score['rouge2'].fmeasure)
    rougeL_scores.append(rouge_score['rougeL'].fmeasure)

# Convert lists of scores to DataFrames
bleu_scores_df = pd.DataFrame(bleu_scores, columns=['bleu_score'])
rouge_scores_df = pd.DataFrame({
    'rouge1_fmeasure': rouge1_scores,
    'rouge2_fmeasure': rouge2_scores,
    'rougeL_fmeasure': rougeL_scores
})

# Print average BLEU and ROUGE scores
average_bleu_score = bleu_scores_df['bleu_score'].mean()
average_rouge1_fmeasure = rouge_scores_df['rouge1_fmeasure'].mean()
average_rouge2_fmeasure = rouge_scores_df['rouge2_fmeasure'].mean()
average_rougeL_fmeasure = rouge_scores_df['rougeL_fmeasure'].mean()

print(f'Average BLEU Score: {average_bleu_score:.4f}')
print(f'Average ROUGE-1 F-measure: {average_rouge1_fmeasure:.4f}')
print(f'Average ROUGE-2 F-measure: {average_rouge2_fmeasure:.4f}')
print(f'Average ROUGE-L F-measure: {average_rougeL_fmeasure:.4f}')

Average BLEU Score: 0.0435
Average ROUGE-1 F-measure: 0.1391
Average ROUGE-2 F-measure: 0.0435
Average ROUGE-L F-measure: 0.1391


In [ ]:
for i in range(10):
  print("1: ", article_corpus[i])
  print("Summary: ", summarize_text(article_corpus[i]))

1:  ब्लैकबेरी हैंडसेट बनाने वाली कनाडा कंपनी रिसर्च मोशन रिम पुनर्गठन योजना तहत अगले सप्ताह वैश्विक स्तर 2 000 कर्मचारियों छंटनी करेगी. द ग्लोब एंड मेल कंपनी नजदीकी सूत्रों हवाले प्रकाशित खबर रिम वैश्विक स्तर बड़े पुनर्गठन तैयारी रही है. अगले सप्ताह वैश्विक स्तर 2 000 नौकरियों कटौती करेगी. रिपोर्ट छंटनी आंकड़ा इससे बड़ा है. द ग्लोब एंड मेल कंपनी नजदीकी सूत्रों हवाले प्रकाशित खबर रिम वैश्विक स्तर बड़े पुनर्गठन तैयारी रही है. अगले सप्ताह वैश्विक स्तर 2 000 नौकरियों कटौती करेगी. रिपोर्ट छंटनी आंकड़ा इससे बड़ा है.
Summary:  करेगी. अगले सप्ताह वैश्विक स्तर 2 000 नौकरियों कटौती करेगी. रिपोर्ट छंटनी आंकड़ा इससे बड़ा है.
1:  अमेरिका राष्ट्रपति पद डेमोक्रेटिक पार्टी उम्मीदवार हिलेरी क्लिंटन निमोनिया पीड़ित पाई गई आराम सलाह दी गई इससे घंटे हिलेरी अस्वस्थ महसूस कारण 9 11 हमलों स्मृति सभा बीच चली गई थीं हिलेरी चिकित्सक लीसा बरडाक बयान पूर्व विदेश मंत्री एलर्जी कारण खांसी रही उन्होंने लंबे समय खांसी रही संबंधी जांच निमोनिया पीड़ित पाया एंटीबॉयोटिक दवाएं देने अलावा आराम कार्यक्रम बदलाव सलाह दी गई चि

In [ ]:
# Save the model to the local filesystem
model.save_pretrained("./fine_tuned_model_epoch_5_final")
tokenizer.save_pretrained("./fine_tuned_model_epoch_5_final")

# Copy the model files to Google Drive
!cp -r ./fine_tuned_model_epoch_5_final /content/drive/MyDrive/